In [1]:
import torch
print(torch.__version__)

1.13.1


## Перцептрон в PyTorch

$x = (x_1, ..., x_n)$ - вектор с данными

$y = (y_1, ..., y_k)$ - то что хотим предсказать, используя x

Сначала попробуем на совсем простой модели:

$\hat{y} = \sigma ( x^T \cdot W + b)$

$$
W =
\begin{pmatrix}
w_{1,1} & w_{1,2} & ... & w_{1,k}\\
...&...&...&...\\
w_{n,1} & w_{n,2} & ... & w_{n,k}\\
\end{pmatrix}
- \text{Матрица весов}\quad ; \quad b =
\begin{pmatrix}
b_{1} & b_{2} & ... & b_{k}\\
\end{pmatrix}
- \text{Вектор смещения}
$$

$$
x^T \cdot W + b =
\begin{pmatrix}
b_1 + \sum_{i=1}^{n} x_i \cdot w_{i,1} & b_2 + \sum_{i=1}^{n} x_i \cdot w_{i,2} & ... & b_k + \sum_{i=1}^{n} x_i \cdot w_{i,k}\\
\end{pmatrix}
$$

$\sigma(a) = \frac{1}{1 + e^{-a}}$ - функция сигмоиды, применяется в нашем случае поэлементно (к каждому элементу вектора по-отдельности)

In [3]:
# Пусть есть вектор с данными x и истинный ответ y_true
x = torch.rand(3)
y_true = torch.tensor([1.])
print(f"x:\n{x}\n\ny_true:\n{y_true}")

x:
tensor([0.7049, 0.3144, 0.1697])

y_true:
tensor([1.])


In [4]:
# Зададим матрицу весов и вектор смещения
# С помощью параметра requires_grad - мы указываем что для данных тензоров нужно будет считать градиенты
w = torch.randn(3, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
print(f"w:\n{w}\n\nb:\n{b}")

w:
tensor([[-2.4852],
        [-0.1744],
        [ 1.1104]], requires_grad=True)

b:
tensor([0.], requires_grad=True)


In [5]:
# Для вычисления сигмоиды возьмём уже готовую из библиотеки torch
torch.sigmoid(torch.tensor([1, 0, -1]))

tensor([0.7311, 0.5000, 0.2689])

In [6]:
# Вычисляем наше предсказание на основании простой модели
y_pred = torch.sigmoid(x.T @ w + b)
y_pred

/var/folders/dn/5x6w91b95lx6tf30sz6djy000000gn/T/ipykernel_8414/1833247472.py:2: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3281.)
  y_pred = torch.sigmoid(x.T @ w + b)


tensor([0.1655], grad_fn=<SigmoidBackward0>)

In [7]:
# Вычисляем наше предсказание на основании простой модели
y_pred = torch.sigmoid(x @ w + b)
y_pred

tensor([0.1655], grad_fn=<SigmoidBackward0>)

In [8]:
# Будем оценивать модуль разницы между оригинальным таргетом y_true и предсказанным y_pred
#  (в более общем случае сумму по всем k, но для примера k=1)
loss = ((y_true - y_pred)**2).sum()
loss

tensor(0.6965, grad_fn=<SumBackward0>)

In [9]:
# Переменная loss уже является просто числом, причём в идеальном ситуации мы именно это число
#  хотим *оптимизировать* - чем меньше это число после очередной итерации, тем лучше наше предсказание
# Будем называть эту переменную "Величина ошибки обучения" или просто "Ошибка"

# Всё что осталось, чтобы сделать один шаг обучения нашей простой модели - обновить матрицу весов 
#  и вектор смещения, используя соответствующие градиенты (градиентный спуск в прошлом модуле)

# Получить градиенты для любой переменной можно используя параметр .grad, однако изначально они пустые
print(
    f"До применения backward",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
    end="\n\n---------\n\n",
)

# Чтобы посчитать градиенты относительно ошибки - функция backward() - используя граф вычислений дойдёт по графу
#  назад до всех переменных с requires_grad = True и посчитает для них градиенты
loss.backward()

print(
    f"После применения backward",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
)

До применения backward

w.grad:
None

b.grad:
None

---------

После применения backward

w.grad:
tensor([[-0.1625],
        [-0.0725],
        [-0.0391]])

b.grad:
tensor([-0.2305])


In [10]:
# Обновим наши веса в соответствии с полученными градиентами (пока без параметра скорости обучения)
print(
    f"До обновления",
    f"w:\n{w}",
    f"b:\n{b}",
    sep="\n\n",
    end="\n\n---------\n\n",
)

with torch.no_grad():
    # Специальная обёртка, чтобы указать что операции внутри не требуют построения графа вычислений
    #  или же подсчёта градиентов
    w -= w.grad  # Важно что делаем через w -= w.grad, а не как w = w - w.grad, потому что иначе мы
    b -= b.grad  #  не меняем текущую переменную, а создаём новую, из-за чего слетят настройки requires_grad = True

print(
    f"После обновления",
    f"w:\n{w}",
    f"b:\n{b}",
    sep="\n\n",
)

До обновления

w:
tensor([[-2.4852],
        [-0.1744],
        [ 1.1104]], requires_grad=True)

b:
tensor([0.], requires_grad=True)

---------

После обновления

w:
tensor([[-2.3227],
        [-0.1019],
        [ 1.1495]], requires_grad=True)

b:
tensor([0.2305], requires_grad=True)


In [11]:
# ОЧЕНЬ ВАЖНО - градиенты не обнуляются, поэтому это нужно делать вручную
#  есть ряд причин, почему это сделано так

print(
    f"До обнуления",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
    end="\n\n---------\n\n",
)

w.grad.zero_()  # in-place обнуление тензора
b.grad.zero_()

print(
    f"После обнуления",
    f"w.grad:\n{w.grad}",
    f"b.grad:\n{b.grad}",
    sep="\n\n",
)

До обнуления

w.grad:
tensor([[-0.1625],
        [-0.0725],
        [-0.0391]])

b.grad:
tensor([-0.2305])

---------

После обнуления

w.grad:
tensor([[0.],
        [0.],
        [0.]])

b.grad:
tensor([0.])
